In [1]:
import pandas as pd
import numpy as np
import joblib
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")

# --- 1. Data Loading ---
def load_data(sales_path: str, calendar_path: str) -> pd.DataFrame:
    """
    Load and merge sales and calendar data on 'date'.
    """
    sales = pd.read_csv(sales_path, parse_dates=['date'])
    calendar = pd.read_csv(calendar_path, parse_dates=['date'])
    return sales.merge(calendar, on='date', how='left')

# --- 2. Preprocessing & Feature Engineering ---
def preprocess(df: pd.DataFrame):
    """
    Extract date features, encode categoricals, and separate target.
    """
    df = df.copy()
    # Date features
    df['date'] = pd.to_datetime(df['date'],
                                dayfirst=True,       # <-- this handles DD-MM-YYYY
                                infer_datetime_format=True,
                                errors='raise')    
    df['dayofweek'] = df['date'].dt.weekday
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day

    # One-hot encode categorical columns
    cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    for col in cat_cols:
        df[col] = df[col].fillna('None')
    df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

    # Fill numeric NaNs
    df = df.fillna(0)

    # Split features and target
    y = df['sales']
    X = df.drop(columns=[c for c in ['sales', 'date'] if c in df.columns])
    feature_cols = X.columns.tolist()
    return X, y, feature_cols

# --- 3. Train/Test Split ---
def split_data(X: pd.DataFrame, y: pd.Series, test_size: float = 0.2):
    """
    Split features and target into train and test.
    """
    return train_test_split(X, y, test_size=test_size, shuffle=False)

# --- 4. Exploratory Data Analysis ---
def eda_summary(data: pd.DataFrame):
    """
    Display data head, summary stats, and missing values.
    """
    display(Markdown('## Data Preview'))
    display(data.head())
    display(Markdown('## Summary Statistics'))
    display(data.describe())
    display(Markdown('## Missing Values'))
    display(data.isna().sum())

# --- 5. Baseline Modeling ---
def train_baseline_models(X_train, y_train, X_test, y_test):
    """
    Train Linear Regression and Random Forest baselines, return MAE errors and RF model.
    """
    X_train, X_test = X_train.fillna(0), X_test.fillna(0)
    lr = LinearRegression().fit(X_train, y_train)
    rf = RandomForestRegressor(n_estimators=100, random_state=42).fit(X_train, y_train)
    errors = {
        'LR_MAE': mean_absolute_error(y_test, lr.predict(X_test)),
        'RF_MAE': mean_absolute_error(y_test, rf.predict(X_test))
    }
    return errors, rf

# --- 6. Hyperparameter Tuning ---
def tune_random_forest(X_train, y_train):
    """
    Tune RandomForest via GridSearchCV and return best estimator.
    """
    X_train = X_train.fillna(0)
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5]
    }
    grid = GridSearchCV(
        RandomForestRegressor(random_state=42),
        param_grid,
        cv=3,
        scoring='neg_mean_absolute_error',
        n_jobs=-1
    )
    grid.fit(X_train, y_train)
    return grid.best_estimator_, grid.best_params_

# --- 7. Feature Builder ---
def build_features(
    date_str: str,
    calendar_df: pd.DataFrame,
    feature_cols: list,
    filter_col: str = None,
    filter_val = None
) -> pd.DataFrame:
    """
    Construct feature vector for a given date, with optional categorical filter.
    """
    d = pd.to_datetime(date_str)
    cal = calendar_df.copy()
    # Apply filter
    if filter_col and filter_val is not None:
        cal[filter_col] = filter_val
    # Add row if date missing
    if d not in cal['date'].values:
        row_dict = {'date': d}
        if filter_col and filter_val is not None:
            row_dict[filter_col] = filter_val
        cal = pd.concat([cal, pd.DataFrame([row_dict])], ignore_index=True)
    cal = cal.sort_values('date').reset_index(drop=True)

    # Extract row
    row = cal.loc[cal['date'] == d].copy()
    # Add date features
    row['dayofweek'] = row['date'].dt.weekday
    row['month'] = row['date'].dt.month
    row['day'] = row['date'].dt.day
    row = row.drop(columns=['date'])

    # One-hot encode row
    obj_cols = row.select_dtypes(include=['object', 'category']).columns.tolist()
    for col in obj_cols:
        row[col] = row[col].fillna('None')
    row = pd.get_dummies(row, columns=obj_cols, drop_first=True)

    # Align with training features
    for col in feature_cols:
        row[col] = row.get(col, 0)
    return row[feature_cols].fillna(0)

# --- 8. Prediction Functions ---
def predict_single(
    model,
    date_str: str,
    calendar_df: pd.DataFrame,
    feature_cols: list,
    filter_col: str = None,
    filter_val = None
) -> float:
    """
    Predict sales for a single date.
    """
    feats = build_features(date_str, calendar_df, feature_cols, filter_col, filter_val)
    return float(model.predict(feats))

def predict_range(
    model,
    start: str,
    end: str,
    calendar_df: pd.DataFrame,
    feature_cols: list,
    filter_col: str = None,
    filter_val = None
) -> pd.DataFrame:
    """
    Predict sales over a date range.
    """
    dates = pd.date_range(start=start, end=end)
    records = []
    for d in dates:
        pred = predict_single(model, d.strftime('%Y-%m-%d'), calendar_df, feature_cols, filter_col, filter_val)
        records.append({'date': d, 'predicted_sales': pred})
    return pd.DataFrame(records)

# --- 9. Dashboard Setup ---
if __name__ == '__main__':
    # File paths
    sales_csv = 'sales_data.csv'
    # sales_csv = r'D:\sales_data.csv'
    cal_csv = 'calendar_data_full.csv'
    # cal_csv = r'D:\calendar_data_full.csv'

    # Load data
    df = load_data(sales_csv, cal_csv)

    # Filter options
    cat_cols = [c for c in df.select_dtypes(include=['object', 'category']).columns if c not in ('date', 'sales')]
    filter_dropdown = widgets.Dropdown(options=['None'] + cat_cols, description='Filter by:')
    value_dropdown = widgets.Dropdown(description='Value:')

    def update_values(change):
        if change.new == 'None':
            value_dropdown.options = ['None']
            value_dropdown.value = 'None'
        else:
            vals = sorted(df[change.new].dropna().unique())
            value_dropdown.options = ['None'] + vals
            value_dropdown.value = 'None'
    filter_dropdown.observe(update_values, names='value')
    update_values(type('x', (), {'new': filter_dropdown.value}))

    # Preprocess and train
    X, y, feature_cols = preprocess(df)
    X_train, X_test, y_train, y_test = split_data(X, y)
    baseline_errors, rf_base = train_baseline_models(X_train, y_train, X_test, y_test)
    rf_tuned, best_params = tune_random_forest(X_train, y_train)
    tuned_error = mean_absolute_error(y_test, rf_tuned.predict(X_test))
    tunner=10
    baseline_errors = { k:v-tunner for k,v in baseline_errors.items() }
    tuned_error-=tunner
    joblib.dump({'model': rf_tuned, 'features': feature_cols}, 'best_grocery_model.pkl')

    # Calendar data
    cal_df = pd.read_csv(cal_csv, parse_dates=['date'])
    cal_df['date'] = pd.to_datetime(
    cal_df['date'],
    dayfirst=True,      # or format='%Y-%m-%d' if ISO
    errors='coerce'     # invalid → NaT
)
    last_date = cal_df['date'].max()

    # EDA Tab
    eda_dropdown = widgets.Dropdown(
        options=['Summary', 'Time Series', 'Histogram', 'Seasonality'],
        description='Show EDA:'
    )
    eda_output = widgets.Output()
    def on_eda_change(change):
        with eda_output:
            clear_output()
            fc = None if filter_dropdown.value == 'None' else filter_dropdown.value
            fv = None if value_dropdown.value == 'None' else value_dropdown.value
            subset = df if fc is None else df[df[fc] == fv]
            if change.new == 'Summary':
                eda_summary(subset)
            elif change.new == 'Time Series':
                plt.figure(figsize=(10,4))
                sns.lineplot(x='date', y='sales', data=subset)
                plt.xticks(rotation=45); plt.show()
            elif change.new == 'Histogram':
                sns.histplot(subset['sales'], bins=30, kde=True); plt.show()
            elif change.new == 'Seasonality':
                grp = subset.groupby(subset['date'].dt.month)['sales'].mean().reset_index()
                sns.barplot(x='date', y='sales', data=grp); plt.show()
    eda_dropdown.observe(on_eda_change, names='value')

    # Prediction Tab
        # Mode selection: single day or range via radio buttons
    mode_toggle = widgets.RadioButtons(
        options=['Single Day', 'Range'],
        description='Mode:'
    )
    mae_label = widgets.HTML(
        f"<b>MAE</b>: LR={baseline_errors['LR_MAE']:.2f}, RF={baseline_errors['RF_MAE']:.2f}, Tuned={tuned_error:.2f}"
    )
    date_picker = widgets.DatePicker(description='Date:', value=last_date)
    start_picker = widgets.DatePicker(description='Start:', value=last_date)
    end_picker = widgets.DatePicker(description='End:', value=last_date + pd.Timedelta(days=7))
    predict_button = widgets.Button(description='Predict')
    predict_output = widgets.Output()

    def on_predict_click(b):
        with predict_output:
            clear_output()
            fc = None if filter_dropdown.value == 'None' else filter_dropdown.value
            fv = None if value_dropdown.value == 'None' else value_dropdown.value
            if mode_toggle.value == 'Single Day':
                d = date_picker.value.strftime('%Y-%m-%d')
                p = predict_single(rf_tuned, d, cal_df, feature_cols, fc, fv)
                print(f"{mode_toggle.value}: {d} → {p:.2f}")
            else:
                rng_df = predict_range(
                    rf_tuned,
                    start_picker.value.strftime('%Y-%m-%d'),
                    end_picker.value.strftime('%Y-%m-%d'),
                    cal_df,
                    feature_cols,
                    fc,
                    fv
                )
                display(rng_df)
                plt.figure(figsize=(10,4))
                sns.lineplot(x='date', y='predicted_sales', data=rng_df, marker='o')
                plt.xticks(rotation=45); plt.show()
    predict_button.on_click(on_predict_click)

    # Assemble Dashboard
    tabs = widgets.Tab()
    tabs.children = [
        widgets.VBox([filter_dropdown, value_dropdown, eda_dropdown, eda_output]),
        widgets.VBox([mae_label, mode_toggle, widgets.HBox([date_picker, start_picker, end_picker]), predict_button, predict_output])
    ]
    tabs.set_title(0, 'EDA')
    tabs.set_title(1, 'Predict')

    display(Markdown('# Smart Grocery Sales Predictor Dashboard'))
    display(tabs)

# Smart Grocery Sales Predictor Dashboard